In [1]:
# Checking Gpu
!nvidia-smi

Wed Oct  7 07:47:13 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# connecting Google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Cloning the DARKNET!
!git clone https://github.com/AlexeyAB/darknet
# Also my repo where I have helper scripts!
!git clone https://github.com/luckyCasualGuy/Scripts.git

Cloning into 'darknet'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 14330 (delta 0), reused 2 (delta 0), pack-reused 14326
Receiving objects: 100% (14330/14330), 12.88 MiB | 3.21 MiB/s, done.
Resolving deltas: 100% (9776/9776), done.
Cloning into 'Scripts'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (40/40), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 40 (delta 10), reused 32 (delta 8), pack-reused 0
Unpacking objects: 100% (40/40), done.


In [ ]:
# configuring & making DARKNET!
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

In [5]:
# Making some required path shortcuts
!ln -s 'cfg/yolov4-tiny-custom.cfg' original_model
!cp original_model 'cfg/yolov4-tiny-model.cfg'
!ln -s 'cfg/yolov4-tiny-model.cfg' original_model_copy
# some shortcuts to my drive
!ln -s '../drive/My Drive' my_drive
!ln -s 'my_drive/OD/dataset' my_data
!ln -s 'my_drive/OD/convs' convs
!ln -s 'my_drive/OD/output' output
# path to my script repo
!ln -s '../Scripts' my_repo
!ln -s 'my_repo/Object detection/' my_scripts
# path to root dir
!ln -s '../' root

 filters=(classes + 5)x3

In [6]:
# !sed -i 's/batch=1/batch=64/' cfg/yolov3-tiny-model.cfg
!sed -i 's/subdivisions=8/subdivisions=64/' cfg/yolov4-tiny-model.cfg
!sed -i 's/max_batches = 500200/max_batches = 10000/' cfg/yolov4-tiny-model.cfg

!sed -i '220 s@classes=80@classes=2@' cfg/yolov4-tiny-model.cfg
!sed -i '269 s@classes=80@classes=2@' cfg/yolov4-tiny-model.cfg
# !sed -i '1144 s@classes=80@classes=2@' cfg/yolov4-tiny-model.cfg

!sed -i '212 s@filters=255@filters=21@' cfg/yolov4-tiny-model.cfg
!sed -i '263 s@filters=255@filters=21@' cfg/yolov4-tiny-model.cfg
# !sed -i '1137 s@filters=255@filters=18@' cfg/yolov4-tiny-model.cfg

In [ ]:
# unzip images in yolo form in the root folder.
!unzip my_data/MASK.zip -d root

In [8]:
# Running script to get obj.data
!python my_scripts/make_obj_3_6.py -d root/images -s root/obj -p 95 -o output

Done! :)


In [12]:
# Start the training
!./darknet detector train root/obj/obj.data original_model_copy output/original_model_copy_last.weights -dont_show

 CUDA-version: 10010 (10010), cuDNN: 7.6.5, GPU count: 1  
 OpenCV version: 3.2.0
original_model_copy
 0 : compute_capability = 370, cudnn_half = 0, GPU: Tesla K80 
net.optimized_memory = 0 
mini_batch = 1, batch = 64, time_steps = 1, train = 1 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 2    416 x 416 x   3 ->  208 x 208 x  32 0.075 BF
   1 conv     64       3 x 3/ 2    208 x 208 x  32 ->  104 x 104 x  64 0.399 BF
   2 conv     64       3 x 3/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.797 BF
   3 route  2 		                       1/2 ->  104 x 104 x  32 
   4 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   5 conv     32       3 x 3/ 1    104 x 104 x  32 ->  104 x 104 x  32 0.199 BF
   6 route  5 4 	                           ->  104 x 104 x  64 
   7 conv     64       1 x 1/ 1    104 x 104 x  64 ->  104 x 104 x  64 0.089 BF
   8 route  2 7 	                           ->  104 x 104 x 128 
   9 max    